##1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). 

##Для унификации предлагаю:

## * взять в тренировочный набор турниры с dateStart из 2019 года; 
## * в тестовый — турниры с dateStart из 2020 года.


In [1]:
import pickle
import io
import zipfile
import pandas as pd
import numpy as np
import warnings
import time
import scipy.stats as sps

from sklearn.linear_model import LogisticRegression
from scipy import sparse

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report as report


In [2]:
warnings.simplefilter('ignore')

# extract the zip file
with zipfile.ZipFile('/content/drive/MyDrive/MADE/advanced ml/chgk.zip') as zip_file:
    # open the first file inside the zip archive
    with zip_file.open(zip_file.namelist()[0], "r") as file:
        # load the pickle data from the file
        players = pickle.load(file)
    with zip_file.open(zip_file.namelist()[1], "r") as file:
        # load the pickle data from the file
        results = pickle.load(file)
    with zip_file.open(zip_file.namelist()[2], "r") as file:
        # load the pickle data from the file
        tournaments = pickle.load(file)

In [4]:
tours = pd.DataFrame(tournaments).T
tours.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [5]:
train_idx = set(tours[tours['dateStart'].apply(lambda x: int(x.split('-')[0])) == 2019].id.values)
test_idx = set(tours[tours['dateStart'].apply(lambda x: int(x.split('-')[0])) == 2020].id.values)

In [6]:
def check_mask(game):
    mask_len = -1
    for team in game:
        if 'mask' not in team:
            return False
        
        
        mask = team['mask']
        if mask is None:
            return False
        if 'X' in mask or '?' in mask:
          return False
        
        team_mask_len = len(mask)
        if mask_len == -1:
            mask_len = team_mask_len
        else:
            if mask_len != team_mask_len:
                return False
    return True

def check_memebers(game):
    for team in game:
        if 'teamMembers' not in team:
            return False
        
        members = team['teamMembers']
        
        if members is None:
            return False
        
        if len(members) == 0:
            return False
    return True

In [7]:
train_json = {key: value for key, value in results.items() if key in train_idx and check_mask(value) and check_memebers(value)}
test_json = {key: value for key, value in results.items() if key in test_idx and check_mask(value)and check_memebers(value)}


In [ ]:
print("train size", len(train_json))
print("test size", len(test_json))

train size 590
test size 367


###Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. 

###Замечания и подсказки:

1.   повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;

2.   в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;

3. для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

  



## Сначала выделим признаки данных. Каждая строка будет содержать следующие поля 'player_id', 'team_id', 'tour_id', 'tour_type', 'quest_num', 'answer':



*   'player_id' - id игрока;
*   'team_id' - id команды;
*    'tour_id' - id турнира;
*    'tour_type' - тип турнира, это значение может говорить о сложности турнира и его вопросов;
*    'quest_num' - номер вопроса в турнире, часто бывает что сложность вопросов возрастает к концу турнира;
*   'answer' - 1/0 (ответил/не ответил) игрок на вопрос


Замечание. Если в маске ответов встречается символ "Х" или "?" мы не выбрасываем турнир, а считаем, что на этот вопрос команда не ответила.



In [ ]:

from scipy import sparse

def json_to_sparse1(df):
  rows = []
  cols = []
  data = []

  i = 0
  j = 0
  n = 0
  for key, value in df.items():
    for team in value:
     for play in team['teamMembers']:
       for n, ans in enumerate(team['mask']):
         rows.append(i)
         cols.append(0)
         data.append(play['player']['id'])

         rows.append(i)
         cols.append(1)
         data.append(j + n)

         rows.append(i)
         cols.append(2)
         data.append(int(ans))

         i += 1
         if i>0 and i%500000 == 0:
           print(f"{i} questions processed")
    j += n

      
  return sparse.csr_matrix((data, (rows, cols)))

col_names = ['player_id', 'quest_num', 'answer']

data_train = json_to_sparse1(train_json)
sparse.save_npz('/content/drive/MyDrive/MADE/advanced ml/train_sparse.npz', data_train)

del data_train

data_test = json_to_sparse1(test_json)
sparse.save_npz('/content/drive/MyDrive/MADE/advanced ml/test_sparse.npz', data_test)

del data_test


500000 questions processed
1000000 questions processed
1500000 questions processed
2000000 questions processed
2500000 questions processed
3000000 questions processed


#Обучим модель логистической регрессии для оценки вероятности ответить на вопрос конкретному игроку. 

Для начала нормализуем наши данные.






In [21]:
ohe = OneHotEncoder(handle_unknown='ignore')

train = sparse.load_npz('/content/drive/MyDrive/MADE/advanced ml/train_sparse.npz')
    
to_keep = [0, 1] 

X_train = train[:,to_keep]
y_train = train.getcol(2)

del train

X_train = ohe.fit_transform(X_train.toarray())


In [22]:
lg = LogisticRegression(max_iter=100)
lg.fit(X_train, np.ravel(y_train.toarray()))


LogisticRegression()

In [ ]:
y_pred = lg.predict(X_train)
print(report(np.ravel(y_train.toarray()), y_pred))

              precision    recall  f1-score   support

           0       0.79      0.82      0.80   7229976
           1       0.75      0.71      0.72   5483836

    accuracy                           0.77  12713812
   macro avg       0.77      0.76      0.76  12713812
weighted avg       0.77      0.77      0.77  12713812



###Для оценки рейтинга игроков найдем предикт модели на тестовых данных и посчитаем для каждого игрока сумму ответов, которые предскажет модель


In [24]:
test = sparse.load_npz('/content/drive/MyDrive/MADE/advanced ml/test_sparse.npz')

to_keep = [0, 1] 

X_test = test[:,to_keep]
y_test = test.getcol(2)


X_test = ohe.transform(X_test.toarray())

y_pred = lg.predict_proba(X_test)[:,1]

In [ ]:
import pandas as pd

df_test = pd.DataFrame(test[:, 0].toarray(), columns=['player_id'])
df_test['ans'] = y_pred

In [ ]:
rating = df_test.groupby('player_id')['ans'].mean().sort_values(ascending=False)
rating.head(10)

player_id
31200     0.910972
4270      0.895421
28751     0.881423
4854      0.876678
151703    0.876678
149111    0.875999
27403     0.873998
174794    0.873531
116753    0.872003
131422    0.855169
Name: ans, dtype: float64

## Топ 20 игроков согласно нашей модели

In [ ]:
name = []
surname = []
user_rating = []

for i, player_id in enumerate(rating.index[:20]):
  name.append(players[player_id]["name"]) 
  surname.append(players[player_id]["surname"])
  user_rating.append(rating.iloc[i])


top_20 = pd.DataFrame()
top_20['player_id'] = rating.index[:20]
top_20["name"] = name
top_20["surname"] = surname
top_20["rating"] = user_rating

top_20

,player_id,name,surname,rating
0,31200,Мария,Танатар,0.910813
1,4270,Александра,Брутер,0.897365
2,149111,Наталья,Каркачёва,0.880233
3,174794,Надежда,Романова,0.879913
4,151703,Артём,Степаненко,0.879086
5,4854,Дмитрий,Вайнштейн,0.879086
6,27403,Максим,Руссо,0.878370
7,28751,Иван,Семушин,0.877664
8,116753,Светлана,Ворошилова,0.861239
9,131422,Игорь,Глик,0.855544


###Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

*  предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;

*  в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.



###Для оценки рейтинга команды в турнире $R_{team}$ для каждого вопроса будем оценивать вероятность ответить командой правильно как вероятность того, что хотя бы один игрок ответил верно: $1- ∏_{i=1}^{N_{team}} q_i$, где $N_{team}$  - число игроков в команде, $q_i = 1 - p_i$  - вероятность, что $i$-й игрок не ответит на вопрос. Таким образом, $R_{team}$  будем рассчитывать как сумму вероятностей ответить на каждый вопрос в турнире:

$R_{team} = \sum_{i=1}^{N_{quest}}\big(1- ∏_{i=1}^{N_{team}} q_i\big)$


In [25]:
test = sparse.load_npz('/content/drive/MyDrive/MADE/advanced ml/test_sparse.npz')

tour_id = []
team_id = []

for key, value in test_json.items():
  for team in value:
    for play in team['teamMembers']:
      for n, ans in enumerate(team['mask']):
         team_id.append(team['team']['id'])
         tour_id.append(key)
         
tours_test = pd.DataFrame(test[:, :2].toarray(), columns=['player_id', 'quest_num'])
tours_test['team_id'] = team_id
tours_test['tour_id'] = tour_id
tours_test['ans'] = y_pred

del test

In [ ]:
tours_test.head(20)

,player_id,quest_num,team_id,tour_id,ans
0,18490,0,66120,5414,0.977991
1,18490,1,66120,5414,0.947348
2,18490,2,66120,5414,0.787476
3,18490,3,66120,5414,0.840212
4,18490,4,66120,5414,0.976919
5,18490,5,66120,5414,0.962439
6,18490,6,66120,5414,0.961834
7,18490,7,66120,5414,0.487431
8,18490,8,66120,5414,0.503895
9,18490,9,66120,5414,0.143282


###Пусть у нас есть баллы набранные командами, отранжированные по убыванию

   $score = [10, 10, 6, 5, 5, 2, 2, 2, 1]$

$r = [1, 2, 3, 4, 5, 6, 7, 8, 9]$

#### Очевидно, что нельзя давать разные ранги командам с одинаковым числом баллов.

#### Связанные ранги будем считать следующим образом:

$R_i^{linked} = \frac{\sum_{i, r_i = v} r_i }{\sum_i (r_i = v)}, \quad  ∀ v \in r, \quad i = \overline{1, N}$

#### Согласно этой формуле новые связанные ранги будут равны

$R^{linked} = [1.5, 1.5, 3, 4.5, 4.5, 7, 7, 7, 9]$

####Таким же образом рассчитаны реальные позиции команд в туре в наших данных.

#### Для расчета $R^{linked}$ можно использовать готовый метод pandas.DataFrame.rank

In [ ]:
def spearman_score(tours_test):
    spearmans = []
    kendalls = []
    ans_col = 'ans'

    for tour in np.unique(tours_test['tour_id']):
        if results[tour] and 'position' in results[tour][0].keys():
            res = tours_test[tours_test['tour_id'] == tour]
            res[ans_col] = 1 - res[ans_col]

            # Calculate the correct answer count for each team and sort them by rank
            team_res = round(res.groupby(['team_id', 'quest_num'], as_index=False)[ans_col].prod())
            team_res['ans_res'] = 1 - team_res[ans_col]
            #print(team_res)
            tour_res = team_res.groupby('team_id')['ans_res'].sum()
            #print(tour_res)

            # Calculate the predicted position for each team
            predicted_position = tour_res.rank(method='min', ascending=False).astype(int)
            #print(predicted_position)

            # Calculate the Spearman and Kendall scores
            team_position = [(team['team']['id'], team['position']) for team in results[tour]]
            
            sorted_pairs = sorted(team_position, key=lambda x: x[0])
            # Extract only the value for each tuple and store in a list
            team_position = [pair[1] for pair in sorted_pairs]
            #print(team_position)
            
            spearman = sps.spearmanr(predicted_position, team_position)[0]
            kendall = sps.kendalltau(predicted_position, team_position)[0]
            #print(spearman, kendall)
            #break
            spearmans.append(spearman)
            kendalls.append(kendall)
        #break

    return np.nanmean(spearmans), np.nanmean(kendalls)


In [ ]:
%%time
print('Spearman, Kendall scores:', spearman_score(tours_test))

Spearman, Kendall scores: (0.7520714834088108, 0.6218486639045641)
CPU times: user 2.14 s, sys: 62.4 ms, total: 2.21 s
Wall time: 1.89 s


##4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
  * предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;

  * разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


In [8]:
import numpy as np


train = sparse.load_npz('/content/drive/MyDrive/MADE/advanced ml/train_sparse.npz')
    

team_ans = []
team_id = []
tour_id = []

for key, value in train_json.items():
  for team in value:
    for play in team['teamMembers']:
      for n, ans in enumerate(team['mask']):
         team_id.append(team['team']['id'])
         team_ans.append(int(ans))
         tour_id.append(key)
         
tours_train = pd.DataFrame(train[:, :2].toarray(), columns=['player_id', 'quest_num'])
tours_train['team_id'] = team_id
tours_train['team_ans'] = team_ans
tours_train['tour_id'] = tour_id


del train




In [ ]:
tours_train.head()

,player_id,quest_num,team_id,team_ans,tour_id
0,6212,0,45556,1,4772
1,6212,1,45556,1,4772
2,6212,2,45556,1,4772
3,6212,3,45556,1,4772
4,6212,4,45556,1,4772


### **E-шаг**: оценка скрытых переменных (правильный/неправильный ответ на вопрос)
  Вероятность правильного ответа на вопрос для каждого игрока вычисляется на основе текущих параметров
  
  Используется формула Байеса: 

  $P_{player}(answer = correct | team) = \frac{P_{player}(answer | correct) * P_{team}(answer| correct)}{P_{team}(answer = correct)}$

  $P_{player}(answer = correct)$ - вероятность правильного ответа для каждого игрока вопрос

  $P_{team}(answer | correct)$ - вероятность, что команда ответила на вопрос верно

  $P_{team}(answer = correct)$ - вероятность команды ответить на вопрос при данном составе игроков

### M-шаг: обновление параметров модели (вероятности правильного ответа на вопрос)
  Новые параметры вычисляются на основе оценок скрытых переменных
  
  Обучаем логистическую регрессию на основе  вероятностей ответов игроков, полученных на E-шаге
   

    

In [ ]:
eps = 1e-8

def E_step(X_train, pred_proba, tours_train1):
    
    # Вычисляем вероятности ответов команд для каждого вопроса
    tours_train1['player_pred_fail'] = 1 - pred_proba
    team_ans_prob = 1 - tours_train1.groupby(['tour_id', 'team_id', 'quest_num'])['player_pred_fail'].prod()
    team_ans_prob = team_ans_prob.rename('team_prob')
    tours_train1 = pd.merge(tours_train1, team_ans_prob, on=['tour_id', 'team_id', 'quest_num'])
    
    # Вычисляем новые вероятности ответов каждого игрока
    tours_train1["new_prob"] = pred_proba / tours_train1['team_prob']
    tours_train1.loc[tours_train1['team_ans'] == 0, "new_prob"] = 0
    tours_train1["new_prob"] = np.clip(tours_train1["new_prob"], eps, 1 - eps)
    player_prob_new = round(tours_train1["new_prob"])
    
    return player_prob_new.tolist()


def M_step(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model


In [ ]:
%%time

import time

iter_num = 10


pred_proba = lg.predict_proba(X_train)[:, 1]

model = LogisticRegression(max_iter=200)

for i in range(iter_num):
  
  start = time.time()
  #i-th E-step
  y_pred = E_step(X_train, pred_proba, tours_train)
  
  # i-th M-step
  model = M_step(model, X_train, y_pred)
  pred_proba = model.predict_proba(X_train)[:, 1]

  # test model
  y_pred_test = model.predict_proba(X_test)[:, 1]
  tours_test['ans'] = y_pred_test
 
  print("Iteration number:", i)
  print("Test score:")
  print("Spearman, Kendall scores:", spearman_score(tours_test))
  print(f"Step time: {(time.time() - start) // 60} min {round((time.time() - start) % 60)} sec")
  print("==================================")
  


Iteration number: 0
Test score:
Spearman, Kendall scores: (0.7650406037535902, 0.6319125982728194)
Step time: 3.0 min 55 sec
Iteration number: 1
Test score:
Spearman, Kendall scores: (0.7511382076940869, 0.6159661140478401)
Step time: 3.0 min 33 sec
Iteration number: 2
Test score:
Spearman, Kendall scores: (0.7580838828056622, 0.6264451750708615)
Step time: 3.0 min 47 sec
Iteration number: 3
Test score:
Spearman, Kendall scores: (0.7558837593780489, 0.6241224657574056)
Step time: 3.0 min 34 sec
Iteration number: 4
Test score:
Spearman, Kendall scores: (0.7505383972265562, 0.6210569016869697)
Step time: 3.0 min 33 sec
Iteration number: 5
Test score:
Spearman, Kendall scores: (0.7418880178502302, 0.6103880811048987)
Step time: 3.0 min 28 sec
Iteration number: 6
Test score:
Spearman, Kendall scores: (0.7477113398054347, 0.6169835702402291)
Step time: 3.0 min 40 sec
Iteration number: 7
Test score:
Spearman, Kendall scores: (0.7445599981427315, 0.613052442655099)
Step time: 3.0 min 38 sec
I

###Как видим улучшения по тестам Спирмана и Кендалла не происходит. Замечено, что при увеличение параметра max_iter логистической регрессии (М-шаг) Spearman score и Kendall score уменьшаются незначительно. При текущем значении max_iter=200 логистическая регрессия не сходится к заданной точности, однако увеличение числа итераций ведет к увеличению времени вычислений, поэтому больше 200 я не пробовала.

###5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).


###Сначала построим рейтинг-лист турниров по сложности, в зависимости от того на какую долю турнира не ответила ни одна команда (в реальности).

In [37]:
test = sparse.load_npz('/content/drive/MyDrive/MADE/advanced ml/test_sparse.npz')

tour_id = []
team_id = []
team_ans = []

for key, value in test_json.items():
  for team in value:
    for play in team['teamMembers']:
      for n, ans in enumerate(team['mask']):
         team_id.append(team['team']['id'])
         team_ans.append(int(ans))
         tour_id.append(key)

         
tours_2020 = pd.DataFrame(test[:, :2].toarray(), columns=['player_id', 'quest_num'])
tours_2020['team_id'] = team_id
tours_2020['tour_id'] = tour_id
tours_2020['team_ans'] = team_ans

del test

In [38]:
# группировка данных по турам и вопросам, подсчет числа ответов на каждый вопрос
questions = tours_2020.groupby(['tour_id', 'quest_num'])['team_ans'].sum().reset_index()

# подсчет числа туров
tours_ids = tours_2020['tour_id'].unique()

del tours_2020

# создание списка для хранения количества вопросов без ответов в каждом туре
unanswered = []
tour_name = []

# перебор туров
for tour in tours_ids:
    # фильтрация данных по текущему туру
    tour_data = questions[questions['tour_id'] == tour]
    # подсчет числа вопросов без ответов в текущем туре
    unanswered.append(len(tour_data[tour_data['team_ans'] == 0]) / len(tour_data))
    tour_name.append(str(tours[tours.id == tour].name.iloc[0]))

# создание датафрейма с рейтингом туров по числу вопросов без ответов
ratings = pd.DataFrame({'tour_id': tours_ids, 'name': tour_name, 'unanswered': unanswered})
ratings = ratings.sort_values('unanswered', ascending=False)

real_top_20_names = list(ratings['name'])
print(ratings.head(20))

del ratings


     tour_id                                               name  unanswered
56      6162                                                 33    0.393939
70      6188  Пятая актава: Тропік Казярога. Ліга нацый: Бел...    0.388889
139     6409                       Онлайн: 08:00 Лига старта-13    0.361111
131     6323                                   Чемпионат Самары    0.208333
107     6256                                        50 на двоих    0.194444
36      6059                       Чемпионат Ростовской области    0.166667
140     6410                           Онлайн: 08:00 Год театра    0.166667
74      6197                    Кубок соседней галактики. Гамма    0.150000
133     6330                                        50 на двоих    0.138889
144     6456                          Онлайн: 16:00 Зелёный шум    0.128205
77      6200                              Студенческие игры БГУ    0.116667
73      6196               Краснодарский рождественский синхрон    0.111111
65      6182

###Теперь составим рейтинг лист для предсказанных вероятностей ответить на вопрос игроками

In [40]:
tour_test_dif = tours_test.copy()

tour_test_dif.loc[tour_test_dif['ans'] < 0.5, 'ans'] = 0
tour_test_dif.loc[tour_test_dif['ans'] > 0.5, 'ans'] = 1

# группировка данных по турам и вопросам, подсчет числа ответов на каждый вопрос
questions = tour_test_dif.groupby(['tour_id', 'quest_num'])['ans'].sum().reset_index()

# подсчет числа туров
tours_ids = tour_test_dif['tour_id'].unique()

del tour_test_dif

# создание списка для хранения количества вопросов без ответов в каждом туре
unanswered = []
tour_name = []

# перебор туров
for tour in tours_ids:
    # фильтрация данных по текущему туру
    tour_data = questions[questions['tour_id'] == tour]
    # подсчет числа вопросов без ответов в текущем туре
    unanswered.append(len(tour_data[tour_data['ans'] == 0]) / len(tour_data))
    tour_name.append(str(tours[tours.id == tour].name.iloc[0]))

# создание датафрейма с рейтингом туров по числу вопросов без ответов
ratings = pd.DataFrame({'tour_id': tours_ids, 'name': tour_name, 'unanswered': unanswered})
ratings = ratings.sort_values('unanswered', ascending=False)

pred_top_20_names = list(ratings['name'])

print(ratings.head(20))

del ratings


     tour_id                                        name  unanswered
136     6340                    Покорение Сибири. День 2    0.688889
144     6456                   Онлайн: 16:00 Зелёный шум    0.666667
141     6428             Онлайн: 19-30 Лёгкий Смоленск-5    0.641026
120     6292                                Зимний Варан    0.615385
107     6256                                 50 на двоих    0.611111
111     6262     Чемпионат Минска. Лига Б. Тур четвёртый    0.611111
34      6045                          Lida Major. День 1    0.583333
106     6250                   Гран-при Бауманки. 3 этап    0.583333
132     6325                                      Буслик    0.583333
139     6409                Онлайн: 08:00 Лига старта-13    0.555556
27      5970                        Январское диминуэндо    0.555556
133     6330                                 50 на двоих    0.527778
64      6181               Кубок Ростова-на-Дону, день 2    0.527778
55      6160                      

### Наша модель правильно определила 7 турниров из топ-20 по сложности:

In [42]:
set(real_top_20_names[:20]) & set(pred_top_20_names[:20])

{'33',
 '50 на двоих',
 'Кубок Ростова-на-Дону, день 2',
 'Кубок соседней галактики. Гамма',
 'Онлайн: 08:00 Лига старта-13',
 'Онлайн: 16:00 Зелёный шум',
 'Чемпионат Самары'}